# TOAR REST-API Demo

In [168]:
from io import StringIO
import json
import pandas as pd
import requests
import time
from pathlib import Path

# Configuration
Insert your access token that can be created in your user profile. A registered user is required for this step.
<br>***Note!** Your access token will be only valid for one hour after creation.*

In [171]:
accessToken = 'eyJraWQiOiI1MTQ0NTM4ODMwNDI3Mjk1NzM2MjI2Mjc4MDM1NTUxMzg3NzY1NDMxNTY4OTIyMjAiLCJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiIzNTU0ZTcyYS02MjUzLTQ2YTktOWUyZS1lMWUzNGM3MGNkNTUiLCJhdWQiOiJ0b2FyX3NlcnZpY2VzIiwic2NvcGUiOiJkaXNwbGF5X25hbWUgb2ZmbGluZV9hY2Nlc3MgZW1haWwgcHJvZmlsZSBlZHVwZXJzb25fc2NvcGVkX2FmZmlsaWF0aW9uIGVkdXBlcnNvbl9lbnRpdGxlbWVudCBlZHVwZXJzb25fdW5pcXVlX2lkIGVkdXBlcnNvbl9hc3N1cmFuY2UiLCJhdXRoX3RpbWUiOjE3NjQ2Nzk5MTAsImlzcyI6Imh0dHBzOi8vbG9naW4uaGVsbWhvbHR6LmRlL29hdXRoMiIsImV4cCI6MTc2NDY4MzkyNSwiaWF0IjoxNzY0Njc5OTI1LCJqdGkiOiI5Nzg0ODEwMC0xZDYyLTQzYmMtYmUwNi1iODE3Mzg1ZWZhY2IiLCJjbGllbnRfaWQiOiJ0b2FyX3NlcnZpY2VzIn0.JI0G_zQNBcBf74-Rt4iMvRNl9C0P5zZkD6CDsXuJJevU1Bk3zeeY_HlR71cSHzRf0MQnPpPeo6nDY9WHINN4su7TtLaLkg0PduY-jeoOB7PhDiiNY48jtgdB1RZH0uXjE0Ed9A-4B4DJruMG95Ad2D5yin61XWhGCtkIA13fEgba_oNVQmuktwWpYvwmmV8xX7f9D_Q_CnS00Xa282ApNWrYrU7naif-HXYhK2FoEfc4P8mhY8p5f52J_QvzmzwNlO822rMkwMFiLog7LR83Ftqe2-pHvUNfX2EEIUCWTS7y_rDzmJTU3NSPOmh6BVlk4nQfMqsPI8s06kgNwAoXKsO6Q8rclGTuDRXY93WD3CLoCLM2iPRN7ue6AxLNvkGcNuNWqGiOlT621cfH7MzS1qK2oCz9WKGK15x0PsQMiZKktvYRmZoq7TeUixeRYryvw5QFGm99nZnUHPNCH4mQBz7VAHCTKD9Ly6xPozz5qttgUKriwGIMjcZ5EQ1xiyo17OzyFPqAIKShRSSlJVoME9R116Jzt_4ViyLwNreauDjU9c9O9Wz4H99C1ARKGpqeqZATENxIbuTHiG1SylBsn0MgvsrK1CuCv9nis7oKndzqWJVnH0eBNF3iY4eRQQ5ePIyxChHUxd_l8uHkiBp_GZ4efHWB1T5JQKfczXv4yGo'

In [179]:
TOAR_SERVICE_URL="https://toar-data.fz-juelich.de/api/v2/"
headers = {'AccessToken': accessToken}
TIME_SERIES_ID = 21
flags = None
file_name = f"toar_time_series_{TIME_SERIES_ID}.csv"

download_folder = Path("data_downloads")
download_folder.mkdir(exist_ok = True)
output_path = download_folder / file_name

# Sample Code from TOAR

In [117]:
# get data from the REST service
flags = None
timeseries_id=21
if flags:
    result=requests.get(TOAR_SERVICE_URL+f'data/timeseries/{timeseries_id}?flags={flags}&format=csv', headers=headers)
else:
    result=requests.get(TOAR_SERVICE_URL+f'data/timeseries/{timeseries_id}?format=csv', headers=headers)
timeseries_meta = json.loads("\n".join([line[1:] for line in result.text.split('\n') if line.startswith('#')]))
df = pd.read_csv(StringIO(result.text), comment='#', header=1, sep=',',names=["time","value","flags","version","timeseries_id"],parse_dates=["time"],index_col="time",date_parser=lambda x:pd.to_datetime(x,utc=True),infer_datetime_format=True)

# print, what I got
print(f"metadata: {timeseries_meta}")
print(f"data: {df}")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [119]:
# example of how to access metadata
stationmeta=timeseries_meta["station"]
variable=timeseries_meta["variable"]
data_start_date = min(df.index).strftime("%Y-%m-%d %H:%M:%S+00")
data_end_date = min(df.index).strftime("%Y-%m-%d %H:%M:%S+00")

# show stationmeta data
print(f'retrieved data from station {stationmeta["name"]} for variable {variable["name"]} with timerange {data_start_date} - {data_end_date}')
print(f'station coordinates: {stationmeta["coordinates"]}')

retrieved data from station DUCKWORTH & ORDINANCE for variable o3 with timerange 2012-01-01 04:30:00+00 - 2012-01-01 04:30:00+00
station coordinates: {'lat': 47.56806, 'lng': -52.70222, 'alt': 7.0}


# Refined Download Script

In [175]:
print(f"Begin with download of time series {TIME_SERIES_ID} ...")

start_time = time.time()

response = requests.get(
    f"{TOAR_SERVICE_URL}data/timeseries/{TIME_SERIES_ID}?format=csv",
    headers = headers
)

end_time = time.time()
elapsed_time = end_time - start_time

# Check that your access token is valid
if response.status_code in(401, 403):
    raise RuntimeError(
        "Access token is missing, invalid, or has expired."
    )

# Check that the given URL and timeseries_id is valid
if response.status_code == 404:
    raise RuntimeError(
        "Requested resource does not exist. Please check for wrong URL or invalid timeseries_id in the configuration above."
    )
    
# Checks that the TOAR database is available
if response.status_code == 503:
    raise RuntimeError(
        "Service is not avaiable. TOAR database may be temporarily offline or overloaded."
    )

print(f"Download finished in {elapsed_time:.2f} seconds.")

# Save the downloaded data on your local drive
print(f"Create folder with name {output_path} and a download file {file_name}.")

with open(output_path, "w", encoding="utf-8") as f:
    f.write(response.text)

print("Done.")


Begin with download of time series 21 ...
Download finished in 0.76 seconds.
Create folder with name data_downloads\toar_time_series_21.csv and a download file toar_time_series_21.csv.
Done.


In [177]:
path = Path("data_downloads")
print("Existiert:", path.exists())
print("Ist Datei:", path.is_file())
print("Ist Ordner:", path.is_dir())

Existiert: True
Ist Datei: False
Ist Ordner: True
